### Fashion MNIST

#### Load training and testing dataset

In [1]:
import mnist_reader
X_train, y_train = mnist_reader.load_mnist('data/fashion', kind='train')
X_test, y_test = mnist_reader.load_mnist('data/fashion', kind='t10k')

#### Convert 1D array of pixels values to 3D array 

In [2]:
from keras import backend as K
img_rows = 28
img_cols = 28
if(K.image_data_format() == 'channels_first'):
    X_train = X_train.reshape(X_train.shape[0], 1, img_rows, img_cols)
    X_test = X_test.reshape(X_test.shape[0], 1, img_rows, img_cols)
    in_shape = (1, img_rows, img_cols)
else:
    X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
    X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)
    in_shape = (img_rows, img_cols, 1)
print(X_train.shape)    
print(X_test.shape)    

Using TensorFlow backend.


(60000, 28, 28, 1)
(10000, 28, 28, 1)


#### One hot encode the target labels

In [3]:
import numpy as np
from keras.utils import np_utils
num_classes = 10
y_train = np_utils.to_categorical(np.array(y_train), num_classes)
y_test = np_utils.to_categorical(np.array(y_test), num_classes)
print(y_train.shape)

(60000, 10)


#### CNN Architecture

In [4]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D

model = Sequential()

model.add(Conv2D(filters=32, kernel_size=3, padding='same', activation='relu', 
                        input_shape=in_shape))
model.add(MaxPooling2D(pool_size=3))
model.add(Conv2D(filters=64, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=3))
model.add(Conv2D(filters=128, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=3))
model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 28, 28, 32)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 9, 9, 32)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 9, 9, 64)          18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 3, 3, 64)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 3, 3, 128)         73856     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 1, 1, 128)         0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 128)               0         
__________

#### Compile the model

In [5]:
model.compile(loss="categorical_crossentropy", optimizer="adadelta", metrics=["accuracy"])

#### Train the model

In [6]:
from keras.callbacks import ModelCheckpoint  

# Save the model with best validation loss
checkpointer = ModelCheckpoint(filepath='best_weights.hdf5', 
                               verbose=1, save_best_only=True)

# Train the model
model.fit(X_train, y_train, epochs=20, batch_size=1000, validation_split=0.2, callbacks=[checkpointer], verbose=1)

Train on 48000 samples, validate on 12000 samples
Epoch 1/20
48000/48000 [==============================] - 40s - loss: 5.0047 - acc: 0.4674 - val_loss: 0.6702 - val_acc: 0.7581
Epoch 2/20
48000/48000 [==============================] - 39s - loss: 0.8309 - acc: 0.7096 - val_loss: 0.5646 - val_acc: 0.7937
Epoch 3/20
48000/48000 [==============================] - 41s - loss: 0.7228 - acc: 0.7434 - val_loss: 0.5601 - val_acc: 0.7932
Epoch 4/20
48000/48000 [==============================] - 41s - loss: 0.6552 - acc: 0.7653 - val_loss: 0.4725 - val_acc: 0.8253
Epoch 5/20
48000/48000 [==============================] - 41s - loss: 0.5971 - acc: 0.7874 - val_loss: 0.4443 - val_acc: 0.8346
Epoch 6/20
48000/48000 [==============================] - 41s - loss: 0.5459 - acc: 0.8066 - val_loss: 0.4281 - val_acc: 0.8387
Epoch 7/20
48000/48000 [==============================] - 43s - loss: 0.5207 - acc: 0.8127 - val_loss: 0.4054 - val_acc: 0.8505
Epoch 8/20
48000/48000 [==============================

#### Load the model with best validation loss

In [7]:
model.load_weights('best_weights.hdf5')

#### Test the model

In [12]:
print("\nAccuracy: ", model.evaluate(X_test, y_test)[1] * 100, "%")

 9856/10000 [============================>.] - ETA: 0s
Accuracy:  88.42 %
